In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
# Read data from file
train_df = pd.read_csv('data/mnist-in-csv/mnist_train.csv', sep=',')
test_df = pd.read_csv('data/mnist-in-csv/mnist_test.csv', sep=',')

train_size = len(train_df)
test_size = len(test_df)

train_data = train_df.iloc[:,1:].to_numpy()
train_target = train_df.iloc[:,0].to_numpy()
train_target = OneHotEncoder(sparse=False).fit_transform(train_target.reshape(-1, 1))

test_data = test_df.iloc[:,1:].to_numpy()
test_target = test_df.iloc[:,0].to_numpy()
test_target = OneHotEncoder(sparse=False).fit_transform(test_target.reshape(-1, 1))

/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mech-user/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'"

In [3]:
train_data = train_data.reshape((-1,28))
test_data = test_data.reshape((-1,28))

In [4]:
train_data.shape

(1680000, 28)

In [5]:
test_data.shape

(280000, 28)

In [6]:
train_target.shape

(60000, 10)

In [7]:
test_target.shape

(10000, 10)

In [8]:
len(train_data[0])

28

In [9]:
train_target = [i for i in train_target for _ in range(len(train_data[0]))]
test_target = [i for i in test_target for _ in range(len(test_data[0]))]

In [10]:
LEAK_RATE=0.2
SPECTRAL_RADIUS = 0.5
#BIAS = 1.0

NUM_INPUT_NODES = 28
NUM_RESERVOIR_NODES = 300
NUM_OUTPUT_NODES = 10

# example of activator
def ReLU(x):
    return np.maximum(0, x)

class ReservoirNetWork: # 入れるtrain_data, test_data, targetは全てnumpy配列

    def __init__(self, num_input_nodes, num_reservoir_nodes, num_output_nodes, leak_rate=0.1, activator=np.tanh):
        self.log_reservoir_nodes = np.array([np.zeros(num_reservoir_nodes)]) # reservoir層のノードの状態を記録

        # init weights
        self.weights_input = self._generate_input_weights(num_input_nodes, num_reservoir_nodes)
        self.weights_reservoir = self._generate_reservoir_weights(num_reservoir_nodes)
        self.weights_output = np.zeros([num_reservoir_nodes, num_output_nodes])

        # それぞれの層のノードの数
        self.num_input_nodes = num_input_nodes
        self.num_reservoir_nodes = num_reservoir_nodes
        self.num_output_nodes = num_output_nodes

        self.leak_rate = leak_rate # 漏れ率
        self.activator = activator # 活性化関数

    # reservoir層のノードの次の状態を取得
    def _get_next_reservoir_nodes(self, input, current_state):
        next_state = (1 - self.leak_rate) * current_state
        next_state += self.leak_rate * (input @ self.weights_input + current_state @ self.weights_reservoir)
        return self.activator(next_state)

    # 出力層の重みを更新
    def _update_weights_output(self, target, lambda0):
        # Ridge Regression
        E_lambda0 = np.identity(self.num_reservoir_nodes) * lambda0
        inv_x = np.linalg.inv(self.log_reservoir_nodes.T @ self.log_reservoir_nodes + E_lambda0)
        # update weights of output layer
        self.weights_output = (inv_x @ self.log_reservoir_nodes.T) @ target

    # 学習する
    def train(self, train_data, lambda0=0.1):
        for i in range(len(train_data)):
            tr = train_data[i]
            
            current_state = np.array(self.log_reservoir_nodes[-1])
            self.log_reservoir_nodes = np.append(self.log_reservoir_nodes, [self._get_next_reservoir_nodes(tr, current_state)], axis=0)
            
            if i % 1000 == 0:
                print('training data no. {}\n'.format(i))
                
        self.log_reservoir_nodes = self.log_reservoir_nodes[1:]
        # まとめて行列計算で重みを更新
        self._update_weights_output(train_target, lambda0)
        
    # 予測する
    def predict(self, test_data):
        predicted_outputs = [np.zeros(3)] # ※1：np.appendをうまく使うためにテキトーな配列を用意、returnする時には除く
        reservoir_nodes = self.log_reservoir_nodes[-1] # 訓練の結果得た最後の内部状態を取得
        for te in test_data:
            reservoir_nodes = self._get_next_reservoir_nodes(te, reservoir_nodes) # 内部状態更新
            output = np.zeros(NUM_OUTPUT_NODES)
            tmp = self.get_output(reservoir_nodes) # 内部状態を読み出して出力を得る
            print(tmp)
            output[ np.argmax(tmp) ] = 1 # 最も出力の値が大きいところが1、残りは0
            predicted_outputs = np.append(predicted_outputs, [output], axis=0)
        return predicted_outputs[1:] # 上の※1参照

    # 内部状態から出力を計算
    def get_output(self, reservoir_nodes):
        return reservoir_nodes @ self.weights_output 

    #############################
    ##### private method ########
    #############################

    # 重みを0.1か0か-0.1で初期化したものを返す
    def _generate_input_weights(self, num_input_nodes, num_reservoir_nodes):
        return np.random.choice([-0.1, 0, 0.1], num_input_nodes*num_reservoir_nodes, p=[0.1, 0.8, 0.1]).reshape(num_input_nodes, num_reservoir_nodes)

    # Reservoir層の重みを初期化
    def _generate_reservoir_weights(self, num_nodes):
        weights = np.random.normal(0, 1, num_nodes * num_nodes)
        indices = np.random.choice(np.arange(len(weights)), replace=False, size=int(len(weights) * 0.9))
        weights[indices] = 0
        weights = weights.reshape([num_nodes, num_nodes])
        spectral_radius = max(abs(linalg.eigvals(weights)))
        return weights / spectral_radius * SPECTRAL_RADIUS


In [ ]:
model = ReservoirNetWork(num_input_nodes=NUM_INPUT_NODES,
    num_reservoir_nodes=NUM_RESERVOIR_NODES,
    num_output_nodes=NUM_OUTPUT_NODES,
    leak_rate=LEAK_RATE)

model.train(train_data) # 訓練

predict_result = model.predict(test_data)



training data no. 0

training data no. 1000

training data no. 2000

training data no. 3000

training data no. 4000

training data no. 5000

training data no. 6000

training data no. 7000

training data no. 8000

training data no. 9000

training data no. 10000

training data no. 11000

training data no. 12000

training data no. 13000

training data no. 14000

training data no. 15000

training data no. 16000

training data no. 17000

training data no. 18000

training data no. 19000

training data no. 20000

training data no. 21000

training data no. 22000

training data no. 23000

training data no. 24000

training data no. 25000

training data no. 26000

training data no. 27000

training data no. 28000



In [ ]:
predict_result

In [ ]:
test_target